#### Data Science Assignment 

Assignment Description

As part of enhancing product understanding, we seek to determine if a product document pertains to a lighting product or not. Your initial task as a Data Scientist is to construct a model for classifying a product PDF page into either a lighting or non-lighting product.


Data Description:
Training Data
The provided training data is a CSV file named `parspec_train_data.csv`, comprising three columns, serving as your training dataset.
ID: PDF ID
URL: The URL where the PDF is hosted; you are required to download the PDFs from this source.
Is Lighting Product? - Gold Label


Test Data: `parspec_test_data.csv`




The Task:
1. Construct a pipeline to extract text from PDFs.
2. Develop a model for predicting the product type, i.e., whether it is a lighting or non-lighting product.
3. Establish an inference pipeline where any user can input a PDF URL, and the pipeline should return the label (lighting or non-lighting) along with class probabilities. This can be achieved by either creating a small function or developing a hosted pipeline.
4. Make predictions on the test data				
						
Deliverables


The following must be submitted:
Code that you wrote to solve the problem
Inference pipeline function or hosted app link 
And, answer to the below questions:
How long did it take to solve the problem?
Explain your solution?
Which model did you use and why?
Any shortcomings and how can we improve the performance?
Report your accuracy on test data
					
				
			
		



In [1]:
import os
import pickle
import requests
from bs4 import BeautifulSoup
from functools import lru_cache
from tqdm import tqdm

import pandas as pd
import numpy as np

# NLP Libraries
import spacy
import nltk
from transformers import AutoTokenizer, AutoModel

# Machine Learning Libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# OpenAI Libraries
import openai

# PDF Processing
import fitz
from PyPDF2 import PdfFileReader

# External Libraries
from llama_cpp import Llama
from dotenv import load_dotenv, find_dotenv


/Users/varshinibalaji/Documents/DSProjects/product_classifier/lclassifier_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [108]:
# load training data
train_data = pd.read_csv("data/parspec_train_data.csv")

In [293]:
train_data.head()

,ID,URL,Is lighting product?
0,P-0,https://a4b6eb24...,Yes
1,P-1,https://a4b6eb24...,Yes
2,P-2,https://myelektr...,Yes
3,P-3,https://myelektr...,Yes
4,P-4,https://myelektr...,Yes


In [14]:
train_data.columns = ['id', 'url', 'is_lighting']

In [15]:
train_data.iloc[0]['url']

'https://a4b6eb24-cef6-4be0-88b1-aa225dd7e4f7.usrfiles.com/ugd/a4b6eb_617bfed66731495e997bcacd789777ad.pdf'

In [16]:
parsed_urls = pd.read_pickle("data/parsed_text_580.pkl")

urls = list(parsed_urls['parsed_text'].to_numpy())

In [17]:
len(urls)

584

In [10]:
from bs4 import BeautifulSoup
import requests
import fitz
import swifter
from tqdm import tqdm

def parse_pdf_from_url(pdf_url):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        pdf_bytes = response.content
        pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")
        # Extract text from each page
        pdf_text = ""
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            pdf_text += page.get_text()
        pdf_document.close()
    else:
        pdf_text = None
    return pdf_text

# parsed_text = parse_pdf_from_url(pdf_url)

train_docs = train_data.to_dict("records")
# urls = []

# for doc in tqdm(train_docs[584:]:
#      urls.append(parse_pdf_from_url(doc['url']))

/Users/varshinibalaji/anaconda3/envs/pdf_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import time

parsed_urls = pd.read_pickle("data/parsed_text_580.pkl")

urls = list(parsed_urls['parsed_text'].to_numpy())

for doc in tqdm(train_docs[584:]):
    try:
     urls.append(parse_pdf_from_url(doc['url']))
    except Exception as e:
        print(e)
        urls.append(None)

  0%|                                           | 1/429 [00:01<09:12,  1.29s/it]

HTTPSConnectionPool(host='cms.buzzi.space', port=443): Max retries exceeded with url: /files/attachments/.10434/6172_Buzzispace_Curtains_Bracks_20210714_LR_WIP.pdf (Caused by SSLError(CertificateError("hostname 'cms.buzzi.space' doesn't match either of '*.l27powered.eu', 'l27powered.eu'")))


  0%|▏                                          | 2/429 [00:02<08:29,  1.19s/it]

HTTPSConnectionPool(host='cms.buzzi.space', port=443): Max retries exceeded with url: /files/attachments/.10505/6209_PSS_BuzziNordic_ST600_v01.pdf (Caused by SSLError(CertificateError("hostname 'cms.buzzi.space' doesn't match either of '*.l27powered.eu', 'l27powered.eu'")))


  1%|▎                                          | 3/429 [00:03<07:29,  1.05s/it]

...

 80%|████████████████████████████████▉        | 344/429 [15:59<04:55,  3.47s/it]

Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


 81%|█████████████████████████████████▎       | 349/429 [16:18<05:38,  4.23s/it]

Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


100%|█████████████████████████████████████████| 429/429 [22:00<00:00,  3.08s/it]


In [34]:
len(urls)

1013

In [35]:
len(train_docs)

1013

In [37]:
df = pd.DataFrame({'parsed_text': urls})

In [44]:
train_data_with_parsed_text = train_data.join(df)

In [45]:
train_data_with_parsed_text.to_pickle("data/train_data_with_parsed_text.pkl")

In [21]:
train_data_with_parsed_text = pd.read_pickle("data/train_data_with_parsed_text.pkl")
train_data_with_parsed_text.shape

(1013, 4)

In [22]:
train_data_with_parsed_text.head()

,id,url,is_lighting,parsed_text
0,P-0,https://a4b6eb24-cef6-4be0-88b1-aa225dd7e4f7.u...,Yes,FEATURES\n•Different color and beam configurat...
1,P-1,https://a4b6eb24-cef6-4be0-88b1-aa225dd7e4f7.u...,Yes,Recommended Use:\n•Architectural\n•Landscaping...
2,P-2,https://myelektralite.com/assets/1018-AI-Outdo...,Yes,None
3,P-3,https://myelektralite.com/assets/1018-Data-She...,Yes,None
4,P-4,https://myelektralite.com/assets/1018AI-Data-S...,Yes,None


cases where the pdf cannot be parsed or parsed_text = None, are cases where the pdf url cannot be opened

In [23]:
# no of links cannot be loaded
len(train_data_with_parsed_text.loc[train_data_with_parsed_text['parsed_text'].isnull()==True])

139

In [24]:
# no of links can be loaded
len(train_data_with_parsed_text.loc[train_data_with_parsed_text['parsed_text'].isnull()==False])

874

In [512]:
# since the no of cases without the parseable pdf link is less fraction, we can drop those records

train_data_with_parsed_text_filtered = train_data_with_parsed_text.loc[train_data_with_parsed_text['parsed_text'].isnull()==False]

train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['is_lighting'] == 'Yes','is_lighting' ] = 1
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['is_lighting'] == 'No','is_lighting' ] = 0

In [513]:
train_data_with_parsed_text_filtered.shape

(874, 4)

In [514]:
train_data_with_parsed_text_filtered.head()

,id,url,is_lighting,parsed_text
0,P-0,https://a4b6eb24...,1,FEATURES\n•Diffe...
1,P-1,https://a4b6eb24...,1,Recommended Use:...
12,P-12,https://en.everl...,0,EVERLIGHT ELECTR...
13,P-13,https://en.everl...,1,\n1 \nCopyrigh...
14,P-14,https://en.everl...,1,\n1 \nCopyrigh...


In [515]:
del train_data_with_parsed_text_filtered['id']

In [516]:
#preprocessing parsed_text
def lowercase(x):
    try:
        return x.lower()
    except Exception:
        return None

train_data_with_parsed_text_filtered['processed_text'] = train_data_with_parsed_text_filtered['parsed_text'].apply(lambda x: lowercase(x))
train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['processed_text'].apply(lambda x: x.split('\n'))

/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/1893469975.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_with_parsed_text_filtered['processed_text'] = train_data_with_parsed_text_filtered['parsed_text'].apply(lambda x: lowercase(x))
/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/1893469975.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['processed_text'].apply(lam

In [517]:
phone_pattern = re.compile(r'([0-9\-\.]+)')
email_pattern = re.compile("([a-zA-Z0-9]+[@][a-zA-Z0-9]+[\.]{1}[a-zA-Z]{2,})")

re.sub(phone_pattern, '', 'phone: 866.248.6300 | 732.486.8520 |  | sales@decorsusa.com')

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/1953798628.py:2: SyntaxWarning: invalid escape sequence '\.'
  email_pattern = re.compile("([a-zA-Z0-9]+[@][a-zA-Z0-9]+[\.]{1}[a-zA-Z]{2,})")


'phone:  |  |  | sales@decorsusacom'

In [518]:
import re

def preprocess_text(lines):
    # Define the set of special characters and numbers to be removed from both ends
    special_chars = ' ,;.!#1234567890'

    # Define a regular expression pattern to match the specified characters at both ends
    updated_lines = []
    for l in lines:
        if 'copyright' in l or 'phone' in l or 'email' in l:
            updated_lines.append('')
        else:
            # Use re.sub to remove the specified characters from both ends
             # Define a pattern to match common website formats
            website_pattern = r'(www\.[a-zA-Z0-9-]+(\.[a-zA-Z]{2,})+)|(http[s]?://[a-zA-Z0-9-]+(\.[a-zA-Z]{2,})+)'
        
            # Combine the patterns to match both special characters at ends and websites
            combined_pattern = f'^[{re.escape(special_chars)}]+|[{re.escape(special_chars)}]+$|{website_pattern}'
        
            # Use re.sub to remove the specified characters and websites from both ends
            processed_text = re.sub(combined_pattern, '', l)
            processed_text = processed_text.strip(' ').strip('.').strip(':').strip('•').strip('°').strip('©').strip('-')

            # removing some stop words observed
            processed_text = processed_text.replace('ltd', '')
            processed_text = processed_text.replace('features', '')
            processed_text = processed_text.replace('co.', '')
            processed_text = processed_text.replace('•', '')
            
            processed_text = re.sub(phone_pattern, '', processed_text)
            processed_text = re.sub(email_pattern, '', processed_text)
            
            remove_special_character_pattern = re.compile(r'[^a-zA-Z0-9\s\.\/,]')
            
            processed_text = re.sub(remove_special_character_pattern, '', processed_text)
            processed_text = processed_text.strip(' ')
            updated_lines.append(processed_text)

    return updated_lines

def remove_special_characters(text):
    # Define a regex pattern to match non-alphanumeric and non-whitespace characters
    pattern = re.compile(r'[^a-zA-Z0-9\s\.\/,]')
    
    # Use the pattern to replace special characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text


train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: preprocess_text(x))

/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/3431438861.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: preprocess_text(x))


In [519]:
def remove_empty(x):
    x = [i for i in x if i != '']
    return x

train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: remove_empty(x))

train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: list(set(x)))

/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/731765168.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: remove_empty(x))
/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/731765168.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambd

In [204]:
train_data_with_parsed_text_filtered.head()

id  \
0    P-0   
1    P-1   
12  P-12   
13  P-13   
14  P-14   

                                                                                                             url  \
0      https://a4b6eb24-cef6-4be0-88b1-aa225dd7e4f7.usrfiles.com/ugd/a4b6eb_617bfed66731495e997bcacd789777ad.pdf   
1      https://a4b6eb24-cef6-4be0-88b1-aa225dd7e4f7.usrfiles.com/ugd/a4b6eb_c53e05a13ae1462eabbdc1c66c1efbb8.pdf   
12  https://en.everlight.com/wp-content/plugins/ItemRelationship/product_files/pdf/031-IRM-37XXT_Series_data.pdf   
13           https://en.everlight.com/wp-content/plugins/ItemRelationship/product_files/pdf/1003SUBD-S400-A6.pdf   
14           https://en.everlight.com/wp-content/plugins/ItemRelationship/product_files/pdf/1003SUGD-S400-A4.pdf   

   is_lighting  \
0          Yes   
1          Yes   
12          No   
13         Yes   
14         Yes   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [520]:
train_data_with_parsed_text_filtered.shape

(874, 5)

In [521]:
# removing - 'fluorescent', 'fluorescents', 'downlights','downlight',
light_stop_words = ['led', 'bulb', 'bulbs', 'xenon', 'filament', 'light', 'lights', 'backlight', 'daylight', 'lumen', 'lamp', 'lamps', 'lumens', 'leds','nightlight',
'lamping', 'neon', 'incandescent',  'fluorescent', 'fluorescents', 'downlights','downlight']

pattern = "(" + light_stop_words[0]
# create a substring pattern

for l in light_stop_words[1:]:
    pattern += "|" + l

pattern += ")"

light_substr_pattern = re.compile(pattern)

    

In [499]:
# using stop words - for inferring light related words - lamp, led, light, 
# checking if a word is a noun
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/varshinibalaji/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [584]:
import time
def check_lighting(parsed_lines):
    fixture_words = []
    for lindex, l in enumerate(parsed_lines):
        # print(f"{parsed_lines=}, {l=}")
        tokens = nltk.word_tokenize(l)
        light_words = list(set(light_stop_words) & set(tokens))
        fw = list(set(fixture_lookaheads) & set(tokens))
        if fw:
            fixture_words = fw
        if light_words:
            pos_tags = nltk.pos_tag(tokens)
            # print(f'{pos_tags=}')
            for tk, tag in pos_tags:
                # print(f"{tk=}, {tag=}")
                if tk in light_words and tag in ['NN', 'NNS', 'JJ']:
                    # if fixture_words == []:
                        return lindex, l, True
    return None, None, False


# train_data_with_parsed_text_filtered['parsed_lines'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: list(x))

train_data_with_parsed_text_filtered['predict_lighting'] = train_data_with_parsed_text_filtered['parsed_lines'].apply(lambda x: check_lighting(x))

In [585]:
train_data_with_parsed_text_filtered['light_phrase_index'] = train_data_with_parsed_text_filtered['predict_lighting'].apply(lambda x: x[0])
train_data_with_parsed_text_filtered['light_phrase'] = train_data_with_parsed_text_filtered['predict_lighting'].apply(lambda x: x[1])
train_data_with_parsed_text_filtered['predict_lighting'] = train_data_with_parsed_text_filtered['predict_lighting'].apply(lambda x: x[2])

In [525]:
fixture_lookaheads = ['socket', 'troffer', 'holder', 'fixture', 'sensor', 'controller', 'transformer']
fixture_lookaheads_plural = []

for f in fixture_lookaheads:
    fixture_lookaheads_plural.append(f+'s')

fixture_lookaheads = fixture_lookaheads_plural+fixture_lookaheads

In [526]:
train_data_with_parsed_text_filtered = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['processed_text'] != '']

In [527]:
train_data_with_parsed_text_filtered.groupby(['predict_lighting', 'is_lighting']).size()

predict_lighting  is_lighting
False             0              237
                  1               26
True              0              170
                  1              408
dtype: int64

#### Correct predictions - TPS + TNS

In [528]:
correct = train_data_with_parsed_text_filtered.loc[(((train_data_with_parsed_text_filtered['predict_lighting'] == False)&(train_data_with_parsed_text_filtered['is_lighting'] == 0))|((train_data_with_parsed_text_filtered['predict_lighting']== True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1)))]
len(correct)

645

In [529]:
len(train_data_with_parsed_text_filtered)

841

#### Understanding performance with only rule based validations and preprocessing

### Accuracy - 77%

In [530]:
len(correct)/len(train_data_with_parsed_text_filtered)

0.7669441141498217

### Precision - 71%

In [531]:
len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1))])/len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['predict_lighting'] == True))])

0.7058823529411765

### Recall/Coverage - 94%

In [532]:
len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1))])/len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['is_lighting'] == 1))])

0.9400921658986175

In [533]:
def check_substr(substr, text):
    if substr in text:
        return True, text
    return False, text

products = list(train_data_with_parsed_text_filtered['processed_text'].to_numpy())
is_lighting = list(train_data_with_parsed_text_filtered['is_lighting'].to_numpy())
urls = list(train_data_with_parsed_text_filtered['url'].to_numpy())

substr = 'lantern'
for index, p in enumerate(products):
    present, match = check_substr(substr, p)
    if present is True:
        print(f"{index=}, {is_lighting[index]=}, {urls[index]=}")
        break

index=166, is_lighting[index]=1, urls[index]='https://www.oldcalifornia.com/files/1002-3-- Small OCL.PDF'


understanding the false positives - i.e. cases where the product is classified as lighting - but it is not

In [534]:
positives = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True]

positives.groupby(['is_lighting', 'light_phrase'], dropna = False).size()

fps = train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 0))]

pd.set_option('display.max_colwidth', 20)
fps.groupby(['is_lighting', 'light_phrase'], dropna = False).size()

is_lighting  light_phrase                                                                                                             
0            ac/dc smoke unit, exit light, smart hush, silent hush                                                                        5
             adjusted for daylight savings time and leap year unless                                                                      1
             and more use the duobrite controller inline with any v to v color changing rgb lights to precisely control any color over    1
             and will be coming out with a troffer light barrier  we also make custom size enclo                                          1
             as the sensor detects an increased light level, the control                                                                  1
                                                                                                                                         ..
             when the sta

#### need clarification - some of the false +ves are not clear to verify

In [254]:
positives.loc[positives['light_phrase'] == 'white led lamps', 'url'].iloc[0]

'https://www.exitlightco.com/pdf/COMBOCA.pdf'

In [277]:
positives.loc[positives['light_phrase'] == 'adjusted for daylight savings time and leap year unless', 'url'].iloc[0]

'https://www.plcmultipoint.com/?wpdmdl=3268'

In [275]:
positives.loc[positives['light_phrase'] == 'with v led drivers, t or t electronic fluorescent ballasts, electronic hid and', 'url'].iloc[0]

'https://cdn.currentlighting.com/site/specsheet/120-277-Transformers.pdf'

In [274]:
positives.loc[positives['light_phrase'] == 'wireless control of color temp adjustable leds', 'url'].iloc[0]

'http://solidapollo.com/PDF/Dynamic-White-GlassTouch-Wall-Controller-120V-AC-Data-Sheet.pdf'

In [256]:
positives.loc[positives['light_phrase'] == 'x center lens led troffer, lumen switchable', 'url'].iloc[0]

'https://cdn.currentlighting.com/site/specsheet/COL_CCL-LSCS_Spec.pdf'

In [258]:
positives.loc[positives['light_phrase'] == 'with e lamp', 'url'].iloc[0]

'https://articolostudios.com/wp-content/uploads/2017/05/Articolo_Studios_Interior_Domi-Wall-Sconce_Spec-Sheet_AU_INT_23-V1.pdf'

In [535]:
positives.loc[positives['light_phrase'] == 'x w medium base sockets for led or cfl bulbs', 'url'].iloc[0]

'https://www.topbrasslighting.com/wp-content/uploads/TopBrass-138.09-tearsheet-Jun12018.pdf'

#### some other kind of products associated with lighting objects but not luminous themselves

- sockets for LED or CFL bulbs.
- Troffers
- product manufactured LANTERN COMPANY
- with a bulb
- without a <\lighting_object>
- w <\lighting_object>
- without <\lighting_object>
- Lamp Holder or Socket
- Lampshade
- lush Mount Fixture
- pendant
- fixture
- SINGLE LEVEL Lighting Controller
- DYNAMIC WHITE GLASSTOUCH 120V AC WIRELESS WALL SWITCH/CONTROLLER
- Step-Up Autotransformers for Lighting Applications
- LIGHTHAWK® PASSIVE INFRARED WALL SWITCH SENSOR
- LCM-IE Interior/Exterior Lighting Control System

In [586]:
train_data_with_parsed_text_filtered = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['processed_text'] != '']
def check_for_negative_lookaheads(x):
    try:
        light_phrase_index = int(x['light_phrase_index'] + 1)
        for lindex, l in enumerate(x['parsed_lines'][:light_phrase_index]):
                tokens = nltk.word_tokenize(l)
                fixture_words = list(set(fixture_lookaheads) & set(tokens))
                if fixture_words:
                    return lindex, l, False
    except Exception as e:
        print(f"{x['parsed_lines']=}, {x['is_lighting']=}, {x['predict_lighting']=}, {x['processed_text']=}, {x['light_phrase']=}")
        print(f"{e=}")
        
    return x['light_phrase_index'], x['light_phrase'], x['predict_lighting']
            

train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True, 'updated_predict_lighting'] = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True].apply(lambda x: check_for_negative_lookaheads(x), axis = 1)

# train_data_with_parsed_text_filtered['updated_predict_lighting'] = train_data_with_parsed_text_filtered.apply(lambda x: check_for_negative_lookaheads(x), axis = 1)
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'light_phrase_index'] = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[0])
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'light_phrase'] = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[1])
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'updated_predict_lighting'] = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[2])

train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == False, 'updated_predict_lighting'] = False

In [537]:
train_data_with_parsed_text_filtered.groupby(['updated_predict_lighting', 'is_lighting'], dropna = False).size()

updated_predict_lighting  is_lighting
False                     0              313
                          1              138
True                      0               94
                          1              296
dtype: int64

In [539]:
positives = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['predict_lighting'] == True]

positives.groupby(['is_lighting', 'light_phrase'], dropna = False).size()

fps = train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['updated_predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 0))]

pd.set_option('display.max_colwidth', 20)
fps.groupby(['is_lighting', 'light_phrase'], dropna = False).size()

is_lighting  light_phrase                                                                                                               
0            ac/dc smoke unit, exit light, smart hush, silent hush                                                                           5
             auto on/ff led work light for dark conditions                                                                                   1
             b  pilot light amber, l  pilot light red, g  pilot light green, u  pilot light blue                                             1
             below the red warning light to access the button, gently pull out                                                               1
             blind to artificial light                                                                                                       1
             bulb included no                                                                                                                4
     

#### updated performance for rule based predictions after additional correction step based on fixture item checks

In [541]:
correct = train_data_with_parsed_text_filtered.loc[(((train_data_with_parsed_text_filtered['updated_predict_lighting'] == False)&(train_data_with_parsed_text_filtered['is_lighting'] == 0))|((train_data_with_parsed_text_filtered['updated_predict_lighting']== True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1)))]

print(f"correct predictions: {len(correct)}")

print(f"total: {len(train_data_with_parsed_text_filtered)}")

print(f"accuracy: {len(correct)/len(train_data_with_parsed_text_filtered)}")

print(f"precision: {len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['updated_predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1))])/len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['updated_predict_lighting'] == True))])}")

print(f"recall: {len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['updated_predict_lighting'] == True)&(train_data_with_parsed_text_filtered['is_lighting'] == 1))])/len(train_data_with_parsed_text_filtered.loc[((train_data_with_parsed_text_filtered['is_lighting'] == 1))])}")

correct predictions: 609
total: 841
accuracy: 0.7241379310344828
precision: 0.7589743589743589
recall: 0.6820276497695853


In [424]:
test_data_final.columns = ['id', 'url', 'is_lighting', 'parsed_text']
test_data_final.to_pickle("data/test_data_final.pkl")

In [429]:
test_data_final = pd.read_pickle("data/test_data_final.pkl")
# test_data_final.shape
test_data_final['processed_text'] = test_data_final['parsed_text'].apply(lambda x: lowercase(x))
test_data_final['parsed_lines'] = test_data_final['processed_text'].apply(lambda x: x.split('\n'))

test_data_final['parsed_lines'] = test_data_final['parsed_lines'].apply(lambda x: preprocess_text(x))
test_data_final['parsed_lines'] = test_data_final['parsed_lines'].apply(lambda x: remove_empty(x))

test_data_final['parsed_lines'] = test_data_final['parsed_lines'].apply(lambda x: list(x))

test_data_final['predict_lighting'] = test_data_final['parsed_lines'].apply(lambda x: check_lighting(x))
test_data_final['light_phrase_index'] = test_data_final['predict_lighting'].apply(lambda x: x[0])
test_data_final['light_phrase'] = test_data_final['predict_lighting'].apply(lambda x: x[1])
test_data_final['predict_lighting'] = test_data_final['predict_lighting'].apply(lambda x: x[2])


test_data_final.loc[test_data_final['predict_lighting'] == True, 'updated_predict_lighting'] = test_data_final.loc[test_data_final['predict_lighting'] == True].apply(lambda x: check_for_negative_lookaheads(x), axis = 1)

# train_data_with_parsed_text_filtered['updated_predict_lighting'] = train_data_with_parsed_text_filtered.apply(lambda x: check_for_negative_lookaheads(x), axis = 1)
test_data_final.loc[test_data_final['predict_lighting'] == True,'light_phrase_index'] = test_data_final.loc[test_data_final['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[0])
test_data_final.loc[test_data_final['predict_lighting'] == True,'light_phrase'] = test_data_final.loc[test_data_final['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[1])
test_data_final.loc[test_data_final['predict_lighting'] == True,'updated_predict_lighting'] = test_data_final.loc[test_data_final['predict_lighting'] == True,'updated_predict_lighting'].apply(lambda x: x[2])

test_data_final.loc[test_data_final['predict_lighting'] == False, 'updated_predict_lighting'] = False

In [543]:
correct = test_data_final.loc[(((test_data_final['predict_lighting'] == False)&(test_data_final['is_lighting'] == 0))|((test_data_final['predict_lighting']== True)&(test_data_final['is_lighting'] == 1)))]

print(f"correct predictions: {len(correct)}")

print(f"total: {len(test_data_final)}")

print(f"accuracy: {len(correct)/len(test_data_final)}")

print(f"precision: {len(test_data_final.loc[((test_data_final['predict_lighting'] == True)&(test_data_final['is_lighting'] == 1))])/len(test_data_final.loc[((test_data_final['predict_lighting'] == True))])}")

print(f"recall: {len(test_data_final.loc[((test_data_final['predict_lighting'] == True)&(test_data_final['is_lighting'] == 1))])/len(test_data_final.loc[((test_data_final['is_lighting'] == 1))])}")

correct predictions: 28
total: 80
accuracy: 0.35
precision: 0.2714285714285714
recall: 0.95


### precision increases - when we include fixture checks in the rule based prediction, but recall drops

In [542]:
correct = test_data_final.loc[(((test_data_final['updated_predict_lighting'] == False)&(test_data_final['is_lighting'] == 0))|((test_data_final['updated_predict_lighting']== True)&(test_data_final['is_lighting'] == 1)))]

print(f"correct predictions: {len(correct)}")

print(f"total: {len(test_data_final)}")

print(f"accuracy: {len(correct)/len(test_data_final)}")

print(f"precision: {len(test_data_final.loc[((test_data_final['updated_predict_lighting'] == True)&(test_data_final['is_lighting'] == 1))])/len(test_data_final.loc[((test_data_final['updated_predict_lighting'] == True))])}")

print(f"recall: {len(test_data_final.loc[((test_data_final['updated_predict_lighting'] == True)&(test_data_final['is_lighting'] == 1))])/len(test_data_final.loc[((test_data_final['is_lighting'] == 1))])}")

correct predictions: 54
total: 80
accuracy: 0.675
precision: 0.4117647058823529
recall: 0.7


In [432]:
train_data_with_parsed_text_filtered.groupby(['updated_predict_lighting', 'is_lighting'], dropna = False).size()

updated_predict_lighting  is_lighting
False                     No             307
                          Yes            123
True                      No             100
                          Yes            311
dtype: int64

In [433]:
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['is_lighting'] == 1, 'is_lighting'] = 1
train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['is_lighting'] == 0, 'is_lighting'] = 0

In [435]:
train_data_with_parsed_text_filtered.columns

Index(['id', 'url', 'is_lighting', 'parsed_text', 'processed_text',
       'parsed_lines', 'predict_lighting', 'light_phrase',
       'light_phrase_index', 'updated_predict_lighting'],
      dtype='object')

In [436]:
test_df.columns

Index(['id', 'url', 'is_lighting', 'parsed_text', 'processed_text',
       'parsed_lines', 'predict_lighting', 'light_phrase_index',
       'light_phrase', 'updated_predict_lighting'],
      dtype='object')

In [544]:
from gensim.models import KeyedVectors
from gensim.downloader import load

# model = load("word2vec-google-news-300")
# word2vec_model.save_word2vec_format("models/word2vec.bin", binary=True)

from gensim.models.keyedvectors import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format("models/word2vec.bin", binary=True)

Text Inference using GPT/LLAMA

below method is used to infer a product description extracted from pdf after processing can be inferred as 

luminous or not, with zero and few-shot prompting

In [361]:
import os
from llama_cpp import Llama
from functools import lru_cache
from dotenv import load_dotenv, find_dotenv
import openai

_ = load_dotenv(find_dotenv())  # read local .env file
_ = load_dotenv('/Users/varshinibalaji/Documents/DSProjects/chatgpt_prompting/config.env')

openai.api_key = os.getenv('OPENAI_API_KEY')

current_path = os.getcwd()

@lru_cache
def load_model(model_name: str = 'llama-2-7b.Q5_K_S.gguf'):  # zephyr-7b-alpha.Q4_K_S
    root = '/Users/varshinibalaji/Documents/DSProjects/' 
    os.chdir(root)

    modelpath = 'models/'+model_name
    llm = Llama(model_path=modelpath)

    os.chdir(current_path)
    return llm


llm = load_model('zephyr-7b-alpha.Q4_K_S.gguf')

@lru_cache
def llama_predict(prompt: str = None):
    output = llm(prompt)
    try:
        return output['choices'][0]['text']
    except Exception:
        return None

parsed_line = "emergency lighting inverters"

prompt = f"""Predict if the given line represents a luminous product: True/False
{parsed_line}
True or False?
"""
# output = llama_predict(prompt)

gpt_prompt = f"""Predict if the given line represents a lighting product: True/False or some irrelevant detail
Examples:
1. Improved immunity against ambient light  - False, reason - it talks about a property of the product and not the product itself,
2. led dual light - True, reason - this clearly represents a lighting product,
3. fully self-contained for plug and go set up - irrelevant, reason - this line describes about the product, but does not hint at what the product is.
Predict if the line below talks about a lighting product:
{parsed_line}
response to return - tuple (True or False, reason)
"""

@lru_cache(maxsize=None)  # Set maxsize to None for an unbounded cache
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# get_completion(gpt_prompt)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/zephyr-7b-alpha.Q4_K_S.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
l

In [121]:
output = get_completion(gpt_prompt)

In [122]:
output

'True, reason - The line specifically mentions "emergency lighting inverters," which indicates that it is talking about a lighting product.'

In [588]:
pd.set_option('display.max_colwidth', 10)

In [590]:
train_data_with_parsed_text_filtered.groupby('light_phrase', dropna = False).size()

light_phrase
,  wire, y/vac, a main, holds  controlled circuits plus  uncontrolled circuits controls up to  breakers  includes lcp controller in      1
a drum shade ceiing pendant fixture made                                                                                                 1
a light that is both minimalist and playful thanks to the                                                                                1
ac/dc smoke unit, exit light, smart hush, silent hush                                                                                    5
actual transformer is enclosed inside the black plastic casing                                                                           5
                                                                                                                                      ... 
x center lens led troffer, lumen switchable                                                                                              1
x e type a  w 

In [587]:
# positives.groupby('is_lighting', dropna = False).size()

train_df = train_data_with_parsed_text_filtered.copy()

train_df['parsed_line_0'] = train_df['parsed_lines'].apply(lambda x: x[0])

train_df.loc[train_df['light_phrase'].isnull()==True, 'light_phrase'] = train_df['parsed_line_0']

In [589]:
train_df.groupby('light_phrase', dropna = False).size()

light_phrase
,  wire, y/vac, a main, holds  controlled circuits plus  uncontrolled circuits controls up to  breakers  includes lcp controller in    1
/ volts dc                                                                                                                             1
a drum shade ceiing pendant fixture made                                                                                               1
a light that is both minimalist and playful thanks to the                                                                              1
a, ah                                                                                                                                  1
                                                                                                                                      ..
x w medium base sockets for led or cfl bulbs                                                                                           1
yes, aluminum is one of the 

### word2vec embedding can be generated for light_phrase as training_feature vector, and used to train a LR model

In [591]:
import spacy
import gensim
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load spaCy and Gensim models
nlp = spacy.load("en_core_web_sm")
# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("path/to/word2vec.model")
word2vec_model = KeyedVectors.load_word2vec_format("models/word2vec.bin", binary=True)

# Define NER function
def extract_lighting_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "ORG" or ent.label_ == "PRODUCT"]
    return entities

# Define function to create word embedding vectors
def text_to_embedding(text):
    words = text.lower().split()
    word_vectors = [word2vec_model.get_vector(word) for word in words if word in word2vec_model]
    return np.mean(word_vectors, axis=0)

#### Training Logistic Regression Model

In [593]:
texts = list(train_df['light_phrase'].to_numpy())

texts

['decors led dual light',
 'connect the fixture cable to the junction box using the',
 'frequency lamps and must be less then  lux at the module surface',
 'are  months if the leds are stored for  months or more, they can be stored for a year in a sealed container with a',
 'are  months if the leds are stored for  months or more, they can be stored for a year in a sealed container with a',
 'are  months if the leds are stored for  months or more, they can be stored for a year in a sealed container with a',
 'are  months if the leds are stored for  months or more, they can be stored for a year in a sealed container with a',
 'are  months if the leds are stored for  months or more, they can be stored for a year in a sealed container with a',
 'if the moisture absorbent material silica gel has faded away or the leds have exceeded the',
 'if the moisture absorbent material silica gel has faded away or the leds have exceeded the',
 'if the moisture absorbent material silica gel has faded aw

In [609]:
train_df  = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['updated_predict_lighting'] == True]
train_df.groupby('is_lighting', dropna = False).size()

is_lighting
0     94
1    296
dtype: int64

In [610]:
94/(296+94)

0.24102564102564103

In [611]:
296/(296+94)

0.7589743589743589

In [619]:
# Load training data
train_df  = train_data_with_parsed_text_filtered.loc[train_data_with_parsed_text_filtered['updated_predict_lighting'] == True]

texts = list(train_df['light_phrase'].to_numpy())
labels = list(train_df['is_lighting'].to_numpy())

# Feature extraction
ner_features = [extract_lighting_entities(text) for text in texts]
ner_features = [",".join(entities) for entities in ner_features]  # Join entities into a single string for simplicity

tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(texts)
embedding_features = [text_to_embedding(text) for text in texts]

# Combine features
features = np.hstack((tfidf_features.toarray(), embedding_features))
class_weight = {0: 0.75, 1: 0.25}  # Assuming 0 is the minority class and 1 is the majority class

# Train the Logistic Regression model
model = LogisticRegression(class_weight=class_weight)
model.fit(features, labels)


LogisticRegression(class_weight={0: 0.75, 1: 0.25})

In [613]:
test_false = test_df.loc[test_df['updated_predict_lighting'] == True]

test_false.groupby('is_lighting', dropna = False).size()

is_lighting
0    20
1    14
dtype: int64

#### testing the accuracy of light classification using word2vec embedding for encoding parsed_text - 44%

In [620]:
# Load test data
test_texts = list(test_false['light_phrase'].to_numpy())

# Feature extraction for test data
test_ner_features = [extract_lighting_entities(text) for text in test_texts]
test_tfidf_features = tfidf_vectorizer.transform(test_texts)
test_embedding_features = [text_to_embedding(text) for text in test_texts]

# Combine features for test data
test_features = np.hstack((test_tfidf_features.toarray(), test_embedding_features))

# Make predictions on the test data
test_predictions = model.predict(test_features)

# Evaluate the accuracy
test_labels = list(test_false['is_lighting'].to_numpy())
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.4411764705882353


In [461]:
test_positives['test_predictions'] = test_predictions

/var/folders/5p/k025km096tndpdfzbd0mhr8c0000gn/T/ipykernel_8019/2685872303.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_positives['test_predictions'] = test_predictions


In [466]:
pd.set_option('display.max_colwidth', None)

In [467]:
test_positives[['light_phrase', 'is_lighting', 'test_predictions']]

,light_phrase,is_lighting,test_predictions
0,"output the system shall be capable of powering any combination of electronic, power factor corrected, fluorescent",0,Yes
1,"output the system shall be capable of powering any combination of electronic, power factor corrected, fluorescent",0,Yes
2,"output the system shall be capable of powering any combination of electronic, power factor corrected, fluorescent",0,Yes
3,lxf / lxt lexington lumen tables,0,Yes
4,lxf / lxt lexington lumen tables,0,Yes
...,...,...,...
75,downlight,0,Yes
76,fixture,0,Yes
77,fixture,0,Yes
78,fixture,0,Yes


In [451]:
# Predict label for a new text
new_text = "white troffers"
new_features = np.hstack((
    tfidf_vectorizer.transform([new_text]).toarray(),
    text_to_embedding(new_text).reshape(1, -1)
))
predicted_label = model.predict(new_features)[0]

# Print the result
print(f"The text '{new_text}' is classified as '{predicted_label}'")

The text 'white troffers' is classified as 'Yes'


In [5]:
# train_data_with_parsed_text_filtered.to_pickle("data/train_data_with_parsed_text_filtered.pkl")
# test_data_final.to_pickle("data/test_data_final.pkl")
import pandas as pd

train_data_with_parsed_text_filtered = pd.read_pickle("data/train_data_with_parsed_text_filtered.pkl")
test_data_final = pd.read_pickle("data/test_data_final.pkl")


train_df = train_data_with_parsed_text_filtered.copy()
test_df = test_data_final.copy()

#### retraining the LR model using bert

In [78]:
from transformers import AutoTokenizer, AutoModel

# Download and load the pre-trained BERT model
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")


In [79]:
import torch

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Flatten the parsed_lines list into a single string
train_texts = train_df["parsed_lines"].apply(lambda x: " ".join(x))
test_texts = test_df["parsed_lines"].apply(lambda x: " ".join(x))

# Feature extraction using TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000)
train_features = vectorizer.fit_transform(train_texts)
test_features = vectorizer.transform(test_texts)

In [128]:
type(bert_model)

transformers.models.bert.modeling_bert.BertModel

In [132]:
type(scaler)

sklearn.preprocessing._data.StandardScaler

In [125]:
type(bert_tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [11]:
# Tokenize each sentence in the parsed lines list
def tokenize_lines(parsed_lines):
    tokenized_lines = []
    for line in parsed_lines:
        tokens = tokenizer(line, truncation=True, padding="max_length", return_tensors="pt")
        tokenized_lines.append(tokens)
    return tokenized_lines

# Apply tokenization
train_bert_features = tokenize_lines(train_texts)
test_bert_features = tokenize_lines(test_texts)

In [81]:
# Extract contextualized word embeddings from BERT
def extract_bert_embeddings(tokenized_lines):
    bert_embeddings = []
    for tokens in tokenized_lines:
        outputs = bert_model(**tokens)
        last_hidden_state = outputs.last_hidden_state
        mean_embedding = torch.mean(last_hidden_state, dim=1)
        bert_embeddings.append(mean_embedding.detach().numpy())
    return bert_embeddings

# # Apply BERT embedding extraction
# train_bert_embeddings = extract_bert_embeddings(train_bert_features)
# test_bert_embeddings = extract_bert_embeddings(test_bert_features)


In [ ]:
# Apply BERT embedding extraction
train_bert_embeddings = extract_bert_embeddings(train_bert_features)
test_bert_embeddings = extract_bert_embeddings(test_bert_features)

In [26]:
# Option 1: Flatten the last two dimensions (words and embedding dimensions)
train_bert_embeddings_flat = [embedding.flatten() for embedding in train_bert_embeddings]
test_bert_embeddings_flat = [embedding.flatten() for embedding in test_bert_embeddings]

train_combined_features = np.concatenate((train_features.toarray(), train_bert_embeddings_flat), axis=1)
test_combined_features = np.concatenate((test_features.toarray(), test_bert_embeddings_flat), axis=1)

In [52]:
from sklearn.linear_model import LogisticRegression
train_labels = train_df['is_lighting']
test_labels = test_df['is_lighting']

In [39]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_df['is_lighting'])
test_labels_encoded = le.transform(test_df['is_lighting'])

# Train the model with encoded labels
model = LogisticRegression()
model.fit(train_features, train_labels_encoded)

LogisticRegression()

In [53]:
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
train_combined_features_scaled = scaler.fit_transform(train_combined_features)
test_combined_features_scaled = scaler.transform(test_combined_features)

le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_df['is_lighting'])
test_labels_encoded = le.transform(test_df['is_lighting'])

# Train the model with scaled features
model = LogisticRegression(max_iter=1000)
model.fit(train_combined_features_scaled, train_labels_encoded)

LogisticRegression(max_iter=1000)

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_combined_features_scaled, train_labels_encoded, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels_encoded, test_size=0.2)

#### Validation accuracy using BERT on training data

In [58]:
model.fit(X_train, y_train)
predictions = model.predict(X_val)

# Calculate accuracy, precision, recall, etc.
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_val, predictions)
precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)

print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")

Validation Accuracy: 0.9763313609467456
Validation Precision: 0.9743589743589743
Validation Recall: 0.9743589743589743


In [59]:
test_predictions = model.predict(test_combined_features_scaled)
# test_predictions = model.predict(test_features)

In [60]:
train_false = train_df.loc[train_df['updated_predict_lighting'] == True]

#### BERT Model Performance on test data - improved to 53% precision, and 77.5% accuracy compared to the use of word2vec embedding

In [106]:
train_false = train_df.loc[train_df['updated_predict_lighting'] == True]
test_texts = test_df["parsed_lines"].apply(lambda x: " ".join(x))
test_features = vectorizer.transform(test_texts)
test_bert_features = tokenize_lines(test_texts)
test_bert_embeddings = extract_bert_embeddings(test_bert_features)
test_bert_embeddings_flat = [embedding.flatten() for embedding in test_bert_embeddings]
test_combined_features = np.concatenate((test_features.toarray(), test_bert_embeddings_flat), axis=1)

# test_labels = 0
# test_labels_encoded = le.transform(test_df['is_lighting'])
test_combined_features_scaled = scaler.transform(test_combined_features)
test_predictions = model.predict(test_combined_features_scaled)

test_labels_encoded = le.transform(test_df['is_lighting'])

accuracy = accuracy_score(test_labels_encoded, test_predictions)
precision = precision_score(test_labels_encoded, test_predictions)
recall = recall_score(test_labels_encoded, test_predictions)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")

Test Accuracy: 0.775
Test Precision: 0.53125
Test Recall: 0.85


In [61]:
accuracy = accuracy_score(test_labels_encoded, test_predictions)
precision = precision_score(test_labels_encoded, test_predictions)
recall = recall_score(test_labels_encoded, test_predictions)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")

Test Accuracy: 0.775
Test Precision: 0.53125
Test Recall: 0.85


In [115]:
# Save the model
import pickle

with open("models/lr_light_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Load the model
with open("models/lr_light_model.pkl", "rb") as f:
    model = pickle.load(f)

In [96]:
def predict_if_lighting(x):
    test_texts = [x]
    test_features = vectorizer.transform(test_texts)
    test_bert_features = tokenize_lines(test_texts)
    test_bert_embeddings = extract_bert_embeddings(test_bert_features)
    test_bert_embeddings_flat = [embedding.flatten() for embedding in test_bert_embeddings]
    test_combined_features = np.concatenate((test_features.toarray(), test_bert_embeddings_flat), axis=1)
    
    # test_labels = 0
    # test_labels_encoded = le.transform(test_df['is_lighting'])
    test_combined_features_scaled = scaler.transform(test_combined_features)
    test_predictions = model.predict(test_combined_features_scaled)
    return True if test_predictions[0] == 1 else False

In [113]:
predict_if_lighting(test_data_final.loc[test_data_final['is_lighting'] == 0, 'parsed_text'].iloc[8])

False